<a href="https://colab.research.google.com/github/EngDiego/colab-testes/blob/main/C%C3%B3pia_de_Copy_of_Copy_of_diffusers_stable_diffusion_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers gradio scipy ftfy "ipywidgets>=7,<8" datasets
!pip install git+https://github.com/pytorch/csprng
!pip install git+https://github.com/huggingface/diffusers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 29.5 MB/s 
     |████████████████████████████████| 6.1 MB 62.0 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 365 kB 71.8 MB/s 
     |████████████████████████████████| 1.6 MB 64.3 MB/s 
     |████████████████████████████████| 6.6 MB 62.2 MB/s 
     |████████████████████████████████| 120 kB 63.0 MB/s 
     |████████████████████████████████| 57 kB 5.3 MB/s 
     |████████████████████████████████| 2.3 MB 56.2 MB/s 
     |████████████████████████████████| 112 kB 74.1 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 212 kB 49.9 MB/s 
     |████████████████████████████████| 84 kB 1.9 MB/s 
     |████████████████████████████████| 275 kB 50.9 MB/s 
     |████████████████████████████████| 5

In [ ]:
rm -rf diffusers

In [ ]:
!git clone https://github.com/huggingface/diffusers.git

Cloning into 'diffusers'...
remote: Enumerating objects: 5868, done.
remote: Counting objects: 100% (293/293), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 5868 (delta 142), reused 201 (delta 101), pack-reused 5575
Receiving objects: 100% (5868/5868), 1.53 MiB | 25.35 MiB/s, done.
Resolving deltas: 100% (3782/3782), done.


In [ ]:
cd diffusers

/content/diffusers


SyntaxError: ignored

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
import gradio as gr
import torch
import torchcsprng as csprng
from torch import autocast
from diffusers import StableDiffusionPipeline, LMSDiscreteScheduler
import requests
from PIL import Image
from io import BytesIO

generator = csprng.create_random_device_generator('/dev/urandom')



from examples.inference.image_to_image import StableDiffusionImg2ImgPipeline, preprocess

lms = LMSDiscreteScheduler(
    beta_start=0.00085, 
    beta_end=0.012, 
    beta_schedule="scaled_linear"
)

#dummy nsfw filter
def dummy_nsfw_safety(images, **kwargs): return images, False

#txt2img
pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", 
    scheduler=lms,
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True
).to("cuda")
pipe.safety_checker = dummy_nsfw_safety


#img2img
pipeimg = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
).to("cuda")
pipeimg.safety_checker = dummy_nsfw_safety

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.63k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/167M [00:00<?, ?B/s]

In [ ]:
block = gr.Blocks(css=".container { max-width: 800px; margin: auto; }")

def infer(prompt, init_image, samples_num, steps_num, scale, strength):
    if init_image != None:
        init_image = init_image.resize((512, 512))
        init_image = preprocess(init_image)
        with autocast("cuda"):
            images = pipeimg([prompt] * samples_num, init_image=init_image, num_inference_steps=steps_num, strength=strength, guidance_scale=scale, generator=generator)["sample"]
    else: 
        with autocast("cuda"):
            images = pipe([prompt] * samples_num,  num_inference_steps=steps_num, strength=strength, guidance_scale=scale, generator=generator)["sample"]

    return images


with block as demo:
    gr.Markdown("<h1><center>Stable Diffusion</center></h1> no image: txt2img")
    with gr.Group():
        with gr.Box():
            with gr.Row().style(mobile_collapse=False, equal_height=True):

                text = gr.Textbox(
                    label="Enter your prompt", show_label=False, max_lines=1
                ).style(
                    border=(True, False, True, True),
                    rounded=(True, False, False, True),
                    container=False,
                )
                btn = gr.Button("Run").style(
                    margin=False,
                    rounded=(False, True, True, False),
                )
        with gr.Row().style(mobile_collapse=False, equal_height=True):
                samples_num = gr.Slider(label="Images", minimum=1, maximum=2, value=2, step=1)
                steps_num = gr.Slider(label="Generation Steps", minimum=1, maximum=200, value=50, step=1)
                scale = gr.Slider(
                   label="CFG Scale", minimum=0, maximum=50, value=7.5, step=0.1
                )
                strength = gr.Slider(
                   label="Strength", minimum=0, maximum=1, value=0.75, step=0.01
                )
        image = gr.Image(
            label="Intial Image",
            type="pil"
        )
               
        gallery = gr.Gallery(label="Generated images", show_label=False).style(
            grid=[2], height="auto"
        )
        text.submit(infer, inputs=[text,image,samples_num,steps_num,scale,strength], outputs=gallery)
        btn.click(infer, inputs=[text,image,samples_num,steps_num,scale,strength], outputs=gallery)


demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://47062.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]